In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vehicle").getOrCreate()

In [10]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://finalchallenge.s3.us-east-2.amazonaws.com/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv"
spark.sparkContext.addFile(url)
all_vehicle_three_year_data_df = spark.read.csv(SparkFiles.get("Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv"), sep=",", header=True, inferSchema=True)
# Show DataFrame
all_vehicle_three_year_data_df.show()

+----+---------------+-----------------+--------------------+------------------------+----------------------------+-------------------+-----------+------------+---------------------+-------------------+----------------+------------+---------------------+---------------------------------+---------------------+---------------------------------+--------------------+-----------------+
|Year|Case Vehicle ID|Vehicle Body Type|  Registration Class|Action Prior to Accident|Type / Axles of Truck or Bus|Direction of Travel|  Fuel Type|Vehicle Year|State of Registration|Number of Occupants|Engine Cylinders|Vehicle Make|Contributing Factor 1|Contributing Factor 1 Description|Contributing Factor 2|Contributing Factor 2 Description|          Event Type|      Partial VIN|
+----+---------------+-----------------+--------------------+------------------------+----------------------------+-------------------+-----------+------------+---------------------+-------------------+----------------+------------+

In [11]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://finalchallenge.s3.us-east-2.amazonaws.com/Vehicle__Snowmobile__and_Boat_Registrations.csv"
spark.sparkContext.addFile(url)
all_vehicle_registration_data_df = spark.read.csv(SparkFiles.get("Vehicle__Snowmobile__and_Boat_Registrations.csv"), sep=",", header=True, inferSchema=True)
# Show DataFrame
all_vehicle_registration_data_df.show()

+-----------+---------------+------------------+--------------+-----+-----+------------+----------+-----+---------+---------+--------------+--------------------+----------+--------------+-------------------+-----+------------------+--------------------+--------------------+
|Record Type|            VIN|Registration Class|          City|State|  Zip|      County|Model Year| Make|Body Type|Fuel Type|Unladen Weight|Maximum Gross Weight|Passengers|Reg Valid Date|Reg Expiration Date|Color|Scofflaw Indicator|Suspension Indicator|Revocation Indicator|
+-----------+---------------+------------------+--------------+-----+-----+------------+----------+-----+---------+---------+--------------+--------------------+----------+--------------+-------------------+-----+------------------+--------------------+--------------------+
|       BOAT|   999999999999|               BOT|           RYE|   NY|10580| WESTCHESTER|      1940|FAIRF|     BOAT|      GAS|          null|                null|      null|   